In [1]:
!pip install -q grobid-client langchain openai faiss-cpu PyPDF2 tiktoken google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.6/229.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 12.9 MB/

In [2]:
import os
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationKGMemory
import requests
import re
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen


os.environ["OPENAI_API_KEY"] =  ""

In [3]:
def create_story(user_input):
    llm = OpenAI(temperature=0.9)

    template = """You are the helpful agent that creates the trailer story that matches the user's input. You need to organize 5 scenes in the context of user input. In user input, the ':' ' symbol, the first is the title and the last is the plot. In Scene 5, You should make the sentence the readers want to know what happens next in story, and the sentence should end with '웹 소설 <제목>에서 확인하실 수 있습니다.'. Don't stray too far from the plot, and don't make things up.
    Example 1)
    user input: 우아한 재혼 : '결혼 조건. 서효주한테는 까맣게 잊을 만큼 쉬운 거였나?' '알아요. 제 역할이 뭐였는지. 그래서 쥐 죽은 듯이 살았다고요. 선배가 원하는 대로 온갖 눈치는 다 보면서, 있는 듯 없는 듯 그렇게 지냈어요. 자그마치 5년씩이나.' 그녀의 첫사랑은 실패였다. 사랑을 갈구했더니 혼인 계약서를 받았고, 시어머니 손에 아이를 유산 당했다. 이혼 후에는 두 번 다시 마주치는 일 따위 없을 줄 알았는데, 낡은 옥탑방으로 전남편이 찾아왔다. '재결합하자.'  '……그럴 마음 없어요.' '제안 아니야. 협박이지.' 얼마나 더 아파야 이 사랑을 끝낼 수 있을까. 얼마나 더 당해야 이 남자의 품에서 벗어날 수 있을까. 효주의 전남편이자 S그룹의 후계자 이명하는 또다시 그녀를 속박하려 했다.
    story: (장면1) 그녀의 첫사랑은 실패했다.\n(장면2) 사랑을 갈구했더니 혼인 계약서를 받았고, 시어머니 손에 아이를 유산 당했다. \n(장면3) 이혼 후 낡은 옥탑방에 사는 그녀를 찾아온 전남편 이명하. \n(장면4) 아프게만 했던 전남편의 첫마디. '재결합하자'\n(장면5) 효주의 전남편이자 S그룹의 후계자 이명하는 또다시 그녀를 속박하려 하는데 둘의 숨막히는 애증 관계가 궁금하시다면, 웹 소설 <우아한 재혼>에서 확인하실 수 있습니다.

    Example 2)
    user input: 회귀자 사용설명서 : 어느 날, 이세계로 소환되었다. 짐승들이 쏟아지고, 믿을 수 없는 위기가 닥쳐오나. 가지고 있는 재능은 밑바닥. '플레이어의 재능 수치는 최하입니다.' '거의 모든 수치가 절망적입니다.' 선택받은 용사든, 재능 있는 마법사들. 시간을 역행한 회귀자든. 모든 것을 이용해야 한다. 살아남기 위해. '쓰레기면 뭐 어떻습니까. 살아남기 위해서 뭔 짓인들 못 하겠어요?'
    story: (장면1) 어느 날, 이세계로 소환되었다.\n(장면2) 짐승들이 쏟아지고, 닥쳐오는 믿을 수 없는 위기\n(장면3) 살아남기 위해 용사, 마법사, 회귀자 모든 것을 이용해야 한다.\n(장면4) '쓰레기면 뭐 어떻습니까. 살아남기 위해서 뭔 짓인들 못 하겠어요?'\n(장면5) 이세계로 소환된 주인공의 살아남는 방법이 궁금하시다면, 웹 소설 <회귀자 사용설명서>에서 확인하실 수 있습니다.

    Relavant Information:
    {history}

    Conversation:
    user input: {input}
    story:"""

    prompt = PromptTemplate(
        input_variables=["history", "input"],
        template=template
    )

    memory = ConversationKGMemory(llm=llm)
    memory.save_context({"input":"우아한 재혼 : '결혼 조건. 서효주한테는 까맣게 잊을 만큼 쉬운 거였나?' '알아요. 제 역할이 뭐였는지. 그래서 쥐 죽은 듯이 살았다고요. 선배가 원하는 대로 온갖 눈치는 다 보면서, 있는 듯 없는 듯 그렇게 지냈어요. 자그마치 5년씩이나.' 그녀의 첫사랑은 실패였다. 사랑을 갈구했더니 혼인 계약서를 받았고, 시어머니 손에 아이를 유산 당했다. 이혼 후에는 두 번 다시 마주치는 일 따위 없을 줄 알았는데, 낡은 옥탑방으로 전남편이 찾아왔다. '재결합하자.'  '……그럴 마음 없어요.' '제안 아니야. 협박이지.' 얼마나 더 아파야 이 사랑을 끝낼 수 있을까. 얼마나 더 당해야 이 남자의 품에서 벗어날 수 있을까. 효주의 전남편이자 S그룹의 후계자 이명하는 또다시 그녀를 속박하려 했다."}, {"output":"(장면1) 그녀의 첫사랑은 실패했다.\n(장면2) 사랑을 갈구했더니 혼인 계약서를 받았고, 시어머니 손에 아이를 유산 당했다. \n(장면3) 이혼 후 낡은 옥탑방에 사는 그녀를 찾아온 전남편 이명하. \n(장면4) 아프게만 했던 전남편의 첫마디. '재결합하자'\n(장면5) 효주의 전남편이자 S그룹의 후계자 이명하는 또다시 그녀를 속박하려 하는데 둘의 숨막히는 애증 관계가 궁금하시다면, 웹 소설 <우아한 재혼>에서 확인하실 수 있습니다."})

    conversation_with_kg = ConversationChain(
        llm=llm,
        verbose=True,
        prompt=prompt,
        memory=memory
    )

    story = conversation_with_kg.predict(input=user_input)

    return story


In [4]:
create_story("환생신의 : '아, 그림자 지면 안 된다고요. 무영등 없어서 못 해 먹겠네.' '무영등이란 무엇이냐?' '그림자 없는 등이요. 수술할 때 필요한데. 말 시키지 말아요.' ‘꼬마 신의! 네가 먼저 말 걸었잖아.’ 하지만 감 아저씨라 불린 자, 백운평은 소리 내어 말하지 않았다. 무영등이라. 그렇다면 이 꼬마의 스승, 정체가 밝혀지지 않은 은거 고수는.. 허공섭물을 자유자재로 구사할 수 있을 정도로 강할 것이다. 등롱을 그림자 없이 들어 올리려면 최소한 공력이 현경의 경지에 달해야 하니까! (아니다. 그냥 환생한 현대인이다. 무영등은 전기로 작동하는 등이거든.)")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
You are the helpful agent that creates the trailer story that matches the user's input. You need to organize 5 scenes in the context of user input. In user input, the ':' ' symbol, the first is the title and the last is the plot. In Scene 5, You should make the sentence the readers want to know what happens next in story, and the sentence should end with '웹 소설 <제목>에서 확인하실 수 있습니다.'. Don't stray too far from the plot, and don't make things up.
    Example 1)
    user input: 우아한 재혼 : '결혼 조건. 서효주한테는 까맣게 잊을 만큼 쉬운 거였나?' '알아요. 제 역할이 뭐였는지. 그래서 쥐 죽은 듯이 살았다고요. 선배가 원하는 대로 온갖 눈치는 다 보면서, 있는 듯 없는 듯 그렇게 지냈어요. 자그마치 5년씩이나.' 그녀의 첫사랑은 실패였다. 사랑을 갈구했더니 혼인 계약서를 받았고, 시어머니 손에 아이를 유산 당했다. 이혼 후에는 두 번 다시 마주치는 일 따위 없을 줄 알았는데, 낡은 옥탑방으로 전남편이 찾아왔다. '재결합하자.'  '……그럴 마음 없어요.' '제안 아니야. 협박이지.' 얼마나 더 아파야 이 사랑을 끝낼 수 있을까. 얼마나 더 당해야 이 남자의 품에서 벗어날 수 있을까. 효주의 전남편이자 S그룹의 후계자 이명하는 또다시 그녀를 속박하려 했다.
    story: (장면1) 그녀의 첫사랑은 실패했다.
(장면2) 사랑을 갈구했더니 혼인 계약서를 받았고, 시어머

" (장면1) 미스터리한 존재인 백운평의 등장\n(장면2) 백운평은 자신만의 특별한 스킬 '무영등'이 필요하다\n(장면3) 백운평을 스승으로 삼은 주인공은 습득한 '무영등' 수술을 시도한다.\n(장면4) 하지만 실패! 그리고 주인공이 자신의 실력을 비하하는 말을 하자 주인공은 씩씩하게 대응한다.\n(장면5) 백운평의 은거 고수로서의 정체와 '무영등'의 정체를 밝히기 위한 주인공의 여정이 궁금하시다면, 웹 소설 <환생신의>에서 확인하실 수 있습니다."